# Prepare input parameters

In [1]:
from osmgt import OsmGt

from osmgt.helpers.plotter import BokehCore

from bokeh.plotting import output_notebook
output_notebook()


location = "Roanne"


# Get POIs

In [2]:
%%time

poi_from_web_found = OsmGt.poi_from_location(location)
poi_gdf = poi_from_web_found.get_gdf()

2020-05-07 22:40:58 - OsmGtPoi        - INFO     : From location: Roanne
2020-05-07 22:40:58 - OsmGtPoi        - INFO     : Loading data...
2020-05-07 22:40:58 - OsmGtPoi        - INFO     : NominatimApi: Query 200:OK in 0:00:00.180886 sec.
2020-05-07 22:41:01 - OsmGtPoi        - INFO     : OverpassApi: Query 200:OK in 0:00:02.802489 sec.
2020-05-07 22:41:01 - OsmGtPoi        - INFO     : Formating data
2020-05-07 22:41:01 - OsmGtPoi        - INFO     : Prepare Geodataframe
CPU times: user 124 ms, sys: 15.5 ms, total: 139 ms
Wall time: 3.11 s


# Get Roads

In [3]:
%%time
roads_from_web_found = OsmGt.roads_from_location(location, poi_gdf)
roads_gdf = roads_from_web_found.get_gdf()

2020-05-07 22:41:01 - OsmGtRoads      - INFO     : From location: Roanne
2020-05-07 22:41:01 - OsmGtRoads      - INFO     : Loading data...
2020-05-07 22:41:01 - OsmGtRoads      - INFO     : NominatimApi: Query 200:OK in 0:00:00.090947 sec.
2020-05-07 22:41:04 - OsmGtRoads      - INFO     : OverpassApi: Query 200:OK in 0:00:02.436475 sec.
2020-05-07 22:41:04 - OsmGtRoads      - INFO     : Formating data
2020-05-07 22:41:04 - OsmGtRoads      - INFO     : Network cleaning STARTS!
2020-05-07 22:41:04 - OsmGtRoads      - INFO     : Starting: Adding new nodes on the network
2020-05-07 22:41:05 - OsmGtRoads      - INFO     : Done: Adding new nodes on the network ; connections_added: 205, line_split: 205
2020-05-07 22:41:05 - OsmGtRoads      - INFO     : Starting: Find intersections
2020-05-07 22:41:05 - OsmGtRoads      - INFO     : Done: Find intersections
2020-05-07 22:41:05 - OsmGtRoads      - INFO     : Starting: build lines
2020-05-07 22:41:06 - OsmGtRoads      - INFO     : Prepare Geoda

# Plot Data

In [59]:
map = BokehCore("My Network Map")

# apply colors based on topology field
roads_gdf_reprojected = roads_gdf[["id", "uuid", "topology", "geometry"]].to_crs(3857)

roads_gdf_unchanged = roads_gdf_reprojected.loc[roads_gdf_reprojected["topology"] == "unchanged"]
roads_gdf_added = roads_gdf_reprojected.loc[roads_gdf_reprojected["topology"] == "added"]
roads_gdf_split = roads_gdf_reprojected.loc[roads_gdf_reprojected["topology"] == "split"]

map.add_multi_lines(
    roads_gdf_unchanged, 
    "roads unchanged",
    "black"
)
map.add_multi_lines(
    roads_gdf_added, 
    "roads added",
    "green"
)
map.add_multi_lines(
    roads_gdf_split, 
    "roads split",
    "orange"
)

map.add_points(
    poi_gdf[["id", "uuid", "geometry"]].to_crs(3857),
    "POIs",
    fill_color="red"
)

# node added on the graph
columns_without_geometry = roads_gdf_added.columns.tolist()
columns_without_geometry.remove("geometry")
roads_gdf_added["geometry"] = roads_gdf_added["geometry"].apply(lambda x: Point(x.coords[-1]))
map.add_points(
    roads_gdf_added[["id", "uuid", "geometry"]],
    "Nodes added",
    fill_color="brown"
)


map.show()

             id  uuid                        geometry
0     260716268     1  POINT (453321.353 5786874.787)
1     260720129     2  POINT (453151.702 5786385.996)
2     260720131     3  POINT (453277.159 5785547.995)
3     260720132     4  POINT (453622.806 5785601.230)
4    1146267475     5  POINT (452232.426 5786562.233)
..          ...   ...                             ...
200  7079559440   201  POINT (454634.032 5787424.548)
201  7399403302   202  POINT (453124.317 5786156.364)
202  7405663025   203  POINT (451430.257 5785722.291)
203  7405663026   204  POINT (451441.167 5785708.822)
204  7405663027   205  POINT (451435.489 5785715.878)

[205 rows x 3 columns]
